In [6]:
adult_fp ← 'adult_sample/adult.csv'
adult ← ⎕CSV adult_fp '' 4
⍴ adult ⍝ Check Size

32562 15

In [7]:
5↑adult

┌───┬────────────┬──────┬───────────┬─────────────┬──────────────────┬───────────────┬─────────────┬──────────────────┬──────┬────────────┬────────────┬──────────────┬──────────────┬──────┐
│age│workclass │fnlwgt│education │education-num│marital-status │occupation │relationship │race │sex │capital-gain│capital-loss│hours-per-week│native-country│salary│
├───┼────────────┼──────┼───────────┼─────────────┼──────────────────┼───────────────┼─────────────┼──────────────────┼──────┼────────────┼────────────┼──────────────┼──────────────┼──────┤
│49 │Private │101320│Assoc-acdm │12 │Married-civ-spouse│ │Wife │White │Female│0 │1902 │40 │United-States │>=50k │
├───┼────────────┼──────┼───────────┼─────────────┼──────────────────┼───────────────┼─────────────┼──────────────────┼──────┼────────────┼────────────┼──────────────┼──────────────┼──────┤
│44 │Private │236746│Masters │14 │Divorced │Exec-managerial│Not-in-family│White │Male │10520 │0 │45 │United-States │>=50k │
├───┼────────────┼──────┼───────────┼─────────────┼──────────────────┼───────────────┼─────────────┼──────────────────┼──────┼────────────┼────────────┼──────────────┼──────────────┼──────┤
│38 │Private │96185 │HS-grad │ │Divorced │ │Unmarried │Black │Female│0 │0 │32 │United-States │<50k │
├───┼────────────┼──────┼───────────┼─────────────┼──────────────────┼───────────────┼─────────────┼──────────────────┼──────┼────────────┼────────────┼──────────────┼──────────────┼──────┤
│38 │Self-emp-inc│112847│Prof-school│15 │Married-civ-spouse│Prof-specialty │Husband │Asian-Pac-Islander│Male │0 │0 │40 │United-States │>=50k │
└───┴────────────┴──────┴───────────┴─────────────┴──────────────────┴───────────────┴─────────────┴──────────────────┴──────┴────────────┴────────────┴──────────────┴──────────────┴──────┘

# Decision Tree

## Get Small Sample

In [8]:
⍝ Get sample of data
col_mask ← ∊ (2⍴1) (12⍴0) 1
adult_sample ← col_mask / 10↑adult
5↑adult_sample

┌───┬────────────┬──────┐
│age│workclass │salary│
├───┼────────────┼──────┤
│49 │Private │>=50k │
├───┼────────────┼──────┤
│44 │Private │>=50k │
├───┼────────────┼──────┤
│38 │Private │<50k │
├───┼────────────┼──────┤
│38 │Self-emp-inc│>=50k │
└───┴────────────┴──────┘

## Encode the matrix

In [9]:
⍝ Get a mask to select a particular column
colnum_mask ← {∊((⍺-1)⍴0) 1 ((⍺-⍨⊣/⍴⍉1↑⍵)⍴0)}
2 colnum_mask adult_sample

0 1 0

In [10]:
⍝ Select a column of data
colloc ← {(⍺ colnum_mask ⍵) / (df←1↓⍵)} 
⎕←col ← 2 colloc adult_sample

┌────────────────┐
│Private │
├────────────────┤
│Private │
├────────────────┤
│Private │
├────────────────┤
│Self-emp-inc │
├────────────────┤
│Self-emp-not-inc│
├────────────────┤
│Private │
├────────────────┤
│Private │
├────────────────┤
│Private │
├────────────────┤
│Private │
└────────────────┘

In [13]:
⍝ Numerical encoding of all columns
encode ← {(uv ⍳ col) (uv ← (≠col) / ,(col←⍺ colloc ⍵))}
encoded ← (⍳3) encode¨ 3⍴⊂adult_sample

⍝ Get data vs keys
⎕←data ← ⊣/¨encoded
⎕←keys ← ⊢/¨encoded

┌───┬───┬───┐
│┌─┐│┌─┐│┌─┐│
││1│││1│││1││
││2│││1│││1││
││3│││1│││2││
││3│││2│││1││
││4│││3│││2││
││5│││1│││2││
││1│││1│││2││
││6│││1│││2││
││7│││1│││1││
│└─┘│└─┘│└─┘│
└───┴───┴───┘

┌──────────────────────┬─────────────────────────────────────────┬──────────────┐
│┌────────────────────┐│┌───────────────────────────────────────┐│┌────────────┐│
││49 44 38 42 20 37 46│││┌───────┬────────────┬────────────────┐│││┌─────┬────┐││
│└────────────────────┘│││Private│Self-emp-inc│Self-emp-not-inc│││││>=50k│<50k│││
│ ││└───────┴────────────┴────────────────┘│││└─────┴────┘││
│ │└───────────────────────────────────────┘│└────────────┘│
└──────────────────────┴─────────────────────────────────────────┴──────────────┘

## Evaluate all splits

In [14]:
⍝ Evaluate all splits for all cols
pos_cor ← {2 1⍴(1-_tmp) (_tmp←∊(⍴y) ÷⍨ +/ (y←∊⊢/⍵-1) = ⍨ (⊂ 1↓(≠ col) / col) < ⍨ (col ← ∊⍺/⍵))}


⎕←pos_acc ← 2 1⍴ (1 0 0) (0 1 0) pos_cor¨ 2⍴⊂data

┌───────────────────────────────────────────────────────────────────────────────┐
│┌─────────────────────────────────────────────────────────────────────────────┐│
││0.5555555556 0.6666666667 0.6666666667 0.5555555556 0.4444444444 0.3333333333││
│├─────────────────────────────────────────────────────────────────────────────┤│
││0.4444444444 0.3333333333 0.3333333333 0.4444444444 0.5555555556 0.6666666667││
│└─────────────────────────────────────────────────────────────────────────────┘│
├───────────────────────────────────────────────────────────────────────────────┤
│┌─────────────────────────┐ │
││0.4444444444 0.5555555556│ │
│├─────────────────────────┤ │
││0.5555555556 0.4444444444│ │
│└─────────────────────────┘ │
└───────────────────────────────────────────────────────────────────────────────┘

## Choose Best Split

In [28]:
1

1

In [29]:
2

2